In [2]:
import os
import pandas as pd
import numpy as np 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
import time
from datetime import datetime
import multiprocessing
import matplotlib.pyplot as plt
import matplotlib
import multiprocessing
matplotlib.use('Agg')
import seaborn as sns
sns.set_style("whitegrid")


# DEVICE SETUP


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_workers = min(8, multiprocessing.cpu_count() - 1)
torch.set_num_threads(4)


print(f"MONTE CARLO DROPOUT LSTM")

print(f"Compute Device:           {device}")
if torch.cuda.is_available():
    print(f"GPU Model:                {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory:               {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"CPU Cores:                {multiprocessing.cpu_count()}")


MONTE CARLO DROPOUT LSTM
Compute Device:           cuda
GPU Model:                NVIDIA GeForce RTX 5060 Ti
GPU Memory:               8.55 GB
CPU Cores:                20


In [4]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True



# LSTM MODEL WITH DROPOUT


In [65]:
class MCDropoutLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=300, num_layers=3, dropout_rate=0.2):
        super(MCDropoutLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        
        # lstm layers with dropout
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.dropout3 = nn.Dropout(dropout_rate)
        
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x, use_dropout=True):
        out, _ = self.lstm1(x)
        if use_dropout:
            out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        if use_dropout:
            out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        if use_dropout:
            out = self.dropout3(out)
        
        out = out[:, -1, :]  
        out = self.fc(out)
        return out



# HELPER FUNCTIONS


In [66]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


def MBE(y_ref, y_test):
    return np.sum(y_test - y_ref) / len(y_ref)


def invTransform(scaler, data, colName, colNames):
    dummy = pd.DataFrame(np.zeros((len(data), len(colNames))), columns=colNames)
    dummy[colName] = data
    dummy = pd.DataFrame(scaler.inverse_transform(dummy), columns=colNames)
    return dummy[colName].values


def calculate_mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return 0.0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0


def calculate_quantile_metrics(y_true, y_pred_p10, y_pred_p50, y_pred_p90):
    y_true = np.array(y_true)
    y_pred_p10 = np.array(y_pred_p10)
    y_pred_p50 = np.array(y_pred_p50)
    y_pred_p90 = np.array(y_pred_p90)
    
    coverage_80 = np.mean((y_true >= y_pred_p10) & (y_true <= y_pred_p90)) * 100
    interval_width = np.mean(y_pred_p90 - y_pred_p10)
    pinaw = interval_width / (np.mean(y_true) + 1e-8)
    
    def quantile_loss(y_true, y_pred, quantile):
        error = y_true - y_pred
        return np.mean(np.maximum(quantile * error, (quantile - 1) * error))
    
    ql_p10 = quantile_loss(y_true, y_pred_p10, 0.1)
    ql_p50 = quantile_loss(y_true, y_pred_p50, 0.5)
    ql_p90 = quantile_loss(y_true, y_pred_p90, 0.9)
    
    return {
        'PICP_80': coverage_80,
        'PINAW': pinaw,
        'QuantileLoss_P10': ql_p10,
        'QuantileLoss_P50': ql_p50,
        'QuantileLoss_P90': ql_p90,
        'AvgQuantileScore': (ql_p10 + ql_p50 + ql_p90) / 3,
        'IntervalWidth': interval_width
    }


In [67]:
def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs=50, patience=10, verbose=True):
    """model with early stopping and progress tracking"""
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    history = {'train_loss': [], 'val_loss': [], 'epoch': []}
    
    epoch_pbar = tqdm(range(num_epochs), desc="Training", disable=not verbose)
    
    for epoch in epoch_pbar:
        # Training phase
        model.train()
        train_loss = 0.0
        
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x, use_dropout=True)
            loss = criterion(outputs, batch_y)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x, use_dropout=False)  
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        history['epoch'].append(epoch + 1)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        
        # progress bar
        epoch_pbar.set_postfix({
            'train': f'{train_loss:.6f}',
            'val': f'{val_loss:.6f}',
            'best_val': f'{best_val_loss:.6f}',
            'patience': f'{patience_counter}/{patience}'
        })
        
        # early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\n Early stopping at epoch {epoch+1}")
                break
    
    # restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, len(history['epoch']), history


def mc_dropout_predict(model, data_loader, n_samples=100, verbose=True):
    model.train()  
    
    all_predictions = []
    
    sample_pbar = tqdm(range(n_samples), desc="MC Sampling", disable=not verbose, leave=False)
    
    for _ in sample_pbar:
        batch_predictions = []
        
        with torch.no_grad(): 
            for batch_x, _ in data_loader:
                batch_x = batch_x.to(device)
                outputs = model(batch_x, use_dropout=True)
                batch_predictions.append(outputs.cpu().numpy())
        
        all_predictions.append(np.vstack(batch_predictions))
    
    return np.array(all_predictions)  

In [68]:
def plot_training_curve(history, save_path):
    """ training and validation loss"""
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    ax.plot(history['epoch'], history['train_loss'], 'b-', label='Training Loss', linewidth=2)
    ax.plot(history['epoch'], history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Loss (MSE)', fontsize=12)
    ax.set_title('Training Progress', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


def plot_mc_predictions(y_true, pred_p10, pred_p50, pred_p90, mc_samples, save_path):
    """ MC Dropout predictions with uncertainty"""
    fig, axes = plt.subplots(3, 1, figsize=(16, 12))
    fig.suptitle('Monte Carlo Dropout Predictions', fontsize=16, fontweight='bold')
    
    n_plot = min(200, len(y_true))
    idx = np.arange(n_plot)
    
    ax = axes[0]
    ax.plot(idx, y_true[:n_plot], 'k-', label='Actual', linewidth=2, alpha=0.8)
    ax.plot(idx, pred_p50[:n_plot], 'b-', label='P50 (Median)', linewidth=1.5)
    ax.fill_between(idx, pred_p10[:n_plot], pred_p90[:n_plot], 
                     alpha=0.3, color='blue', label='P10-P90 Interval (80%)')
    
    outside = (y_true[:n_plot] < pred_p10[:n_plot]) | (y_true[:n_plot] > pred_p90[:n_plot])
    if outside.sum() > 0:
        ax.scatter(idx[outside], y_true[:n_plot][outside], 
                  color='red', s=50, marker='x', label='Outside Interval', zorder=5)
    
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('GHI (W/m²)')
    ax.set_title(f'Predictions with Uncertainty (first {n_plot} samples)')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    ax = axes[1]
    n_show = min(20, mc_samples.shape[0])
    sample_indices = np.random.choice(mc_samples.shape[0], n_show, replace=False)
    
    for i in sample_indices:
        ax.plot(idx, mc_samples[i, :n_plot, 0], alpha=0.3, linewidth=0.8, color='gray')
    
    ax.plot(idx, y_true[:n_plot], 'k-', label='Actual', linewidth=2, alpha=0.8)
    ax.plot(idx, pred_p50[:n_plot], 'r-', label='Median', linewidth=2)
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('GHI (W/m²)')
    ax.set_title(f'MC Dropout Samples (showing {n_show} of 100 stochastic passes)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    ax = axes[2]
    ax2 = ax.twinx()
    
    interval_width = (pred_p90 - pred_p10)[:n_plot]
    residuals = y_true[:n_plot] - pred_p50[:n_plot]
    
    ax.plot(idx, residuals, 'b-', alpha=0.6, label='Residuals', linewidth=1.5)
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)
    ax2.plot(idx, interval_width, 'r-', alpha=0.6, linewidth=1.5, label='Interval Width')
    
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('Residual (W/m²)', color='b')
    ax2.set_ylabel('Interval Width (W/m²)', color='r')
    ax.set_title('Prediction Error vs Uncertainty')
    ax.tick_params(axis='y', labelcolor='b')
    ax2.tick_params(axis='y', labelcolor='r')
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


def plot_coverage_analysis(y_true, pred_p10, pred_p50, pred_p90, save_path):
    """Plot coverage analysis"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('Coverage & Calibration Analysis', fontsize=16, fontweight='bold')
    
    in_interval = (y_true >= pred_p10) & (y_true <= pred_p90)
    interval_width = pred_p90 - pred_p10
    coverage = in_interval.mean() * 100
    
    ax = axes[0, 0]
    bins = np.percentile(y_true, [0, 20, 40, 60, 80, 100])
    digitized = np.digitize(y_true, bins)
    coverage_by_bin = [in_interval[digitized == i].mean() * 100 for i in range(1, len(bins))]
    x_labels = [f'{bins[i]:.0f}-{bins[i+1]:.0f}' for i in range(len(bins)-1)]
    
    bars = ax.bar(range(len(coverage_by_bin)), coverage_by_bin, alpha=0.7)
    ax.axhline(y=80, color='red', linestyle='--', linewidth=2, label='Target (80%)')
    ax.set_xlabel('GHI Range (W/m²)')
    ax.set_ylabel('Coverage (%)')
    ax.set_title('Coverage by Value Range')
    ax.set_xticks(range(len(coverage_by_bin)))
    ax.set_xticklabels(x_labels, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    for i, bar in enumerate(bars):
        if 75 <= coverage_by_bin[i] <= 85:
            bar.set_color('green')
        elif coverage_by_bin[i] < 70:
            bar.set_color('red')
        else:
            bar.set_color('orange')
    
    ax = axes[0, 1]
    ax.hist(interval_width, bins=50, alpha=0.7, edgecolor='black')
    ax.axvline(x=interval_width.mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {interval_width.mean():.1f}')
    ax.set_xlabel('Interval Width (W/m²)')
    ax.set_ylabel('Frequency')
    ax.set_title('Distribution of Uncertainty')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    ax = axes[1, 0]
    scatter = ax.scatter(y_true, interval_width, c=in_interval, 
                        cmap='RdYlGn', alpha=0.5, s=10)
    ax.set_xlabel('Actual GHI (W/m²)')
    ax.set_ylabel('Interval Width (W/m²)')
    ax.set_title('Interval Width vs Actual Value')
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='In Interval')
    
    ax = axes[1, 1]
    ax.axis('off')
    
    below_p10 = (y_true < pred_p10).sum()
    above_p90 = (y_true > pred_p90).sum()
    
    summary = f"""Overall Coverage:     {coverage:.2f}%"""
    
    ax.text(0.05, 0.5, summary, fontsize=10, family='monospace',
            verticalalignment='center', transform=ax.transAxes,
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


# CONFIGURATION


In [69]:
nbEpoch = 50
batch_size = 256
learning_rate = 0.001
patience = 10
mc_samples = 100  
dropout_rate = 0.2


#just key time horizons
nPrevSteps_list = [10, 20] 


polynomialAugm_list = [1, 2]  

samplingFrequencies_eng = ["15_minutes", "30_minutes", "1_hour"] 

basicFeatures = ['GHI', 'GHIcs', 'k']
seasonalFeatures = ['month', 'day', 'hour']
d1Features = ['GHI_d1', 'GHIcs_d1', 'k_d1']
d2Features = ['GHI_d2', 'GHIcs_d2', 'k_d2']


featureColumns_list = [
    basicFeatures,                           
    basicFeatures + d1Features,             
    basicFeatures + seasonalFeatures,        
    basicFeatures + seasonalFeatures + d1Features 
]

featureColumns_eng_list = ['GHI', 'GHI_d1', 'GHI_season', 'GHI_season_d1']


data_dir = './GHI_dataset/cleaned_sampled_data/'
reportLSTM_dir = './reports/LSTM_mcdropout/'

# MAIN LOOP


In [70]:
total_configs = len(samplingFrequencies_eng) * len(featureColumns_list) * len(polynomialAugm_list) * len(nPrevSteps_list)
current_config = 0
start_time = time.time()

print(f"Total configurations: {total_configs}")

for freq_idx, samplingFrequency in enumerate(samplingFrequencies_eng):
    print("\n" + "="*80)
    print(f"Frequency: {samplingFrequency} ({freq_idx+1}/{len(samplingFrequencies_eng)})")
    print("="*80)
    
    report_dir = os.path.join(reportLSTM_dir, samplingFrequency)
    models_dir = os.path.join(report_dir, 'models')
    plots_dir = os.path.join(report_dir, 'plots')
    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)
    
    file_path = os.path.join(data_dir, f'GHI_sampled_{samplingFrequency}.csv')
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    
    data_df = pd.read_csv(file_path, index_col=0)
    print(f"Loaded: {data_df.shape[0]} samples")
    
    summary_results = []
    
    for k, featureColumns in enumerate(featureColumns_list):
        print(f"\nFeatures: {featureColumns_eng_list[k]} ({k+1}/{len(featureColumns_list)})")
        
        for polynomialAug in polynomialAugm_list:
            for nPrevSteps in nPrevSteps_list:
                current_config += 1
                elapsed = time.time() - start_time
                eta = (elapsed / current_config) * (total_configs - current_config)
                
                print(f"  [{current_config}/{total_configs}] Poly={polynomialAug}, Hist={nPrevSteps} | ETA: {eta/60:.1f}min")

                
                set_seed(42)
                X = data_df.loc[:, featureColumns]
                
                poly = PolynomialFeatures(polynomialAug)
                X_poly = poly.fit_transform(X)
                colNames = poly.get_feature_names_out(X.columns)
                
                scaler = MinMaxScaler(feature_range=(0, 1))
                scaled = scaler.fit_transform(X_poly)
                scaled = pd.DataFrame(scaled, columns=colNames, index=data_df.index)
                scaled = scaled.drop("1", axis=1)
                nFeatureColumns = scaled.shape[1]
                
                train_test_df = pd.DataFrame()
                for i in range(nPrevSteps + 1):
                    title = scaled.columns + 't(-' + str(i) + ')'
                    temp = scaled.shift(periods=i)
                    temp.columns = title
                    train_test_df = pd.concat([train_test_df, temp], axis=1)
                
                train_test_df = train_test_df.dropna()
                
                title_0 = scaled.columns + 't(-' + str(0) + ')'
                X = train_test_df.drop(title_0, axis=1).values
                y = train_test_df['GHIt(-0)'].values.reshape(-1, 1)
                
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
                X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=False)
                
                X_train_r = X_train.reshape((X_train.shape[0], nPrevSteps, nFeatureColumns))
                X_val_r = X_val.reshape((X_val.shape[0], nPrevSteps, nFeatureColumns))
                X_test_r = X_test.reshape((X_test.shape[0], nPrevSteps, nFeatureColumns))
                
                train_dataset = TensorDataset(torch.FloatTensor(X_train_r), torch.FloatTensor(y_train))
                val_dataset = TensorDataset(torch.FloatTensor(X_val_r), torch.FloatTensor(y_val))
                test_dataset = TensorDataset(torch.FloatTensor(X_test_r), torch.FloatTensor(y_test))
                
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False,
                                         num_workers=num_workers, pin_memory=True if torch.cuda.is_available() else False)
                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                                       num_workers=num_workers, pin_memory=True if torch.cuda.is_available() else False)
                test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                                        num_workers=num_workers, pin_memory=True if torch.cuda.is_available() else False)
                
                # TRAIN MODEL
                print(f"\n Training MC Dropout model...")
                
                model = MCDropoutLSTM(
                    input_size=nFeatureColumns,
                    hidden_size=300,
                    num_layers=3,
                    dropout_rate=dropout_rate
                ).to(device)
                
                criterion = nn.MSELoss()
                optimizer = optim.Adam(model.parameters(), lr=learning_rate)
                
                model, epochs_trained, history = train_model(
                    model, train_loader, val_loader,
                    criterion, optimizer, nbEpoch, patience, verbose=True
                )
                
                print(f" Model trained ({epochs_trained} epochs)")
                
                # Save model
                model_filename = f'model_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.pt'
                model_path = os.path.join(models_dir, model_filename)
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'input_size': nFeatureColumns,
                    'hidden_size': 300,
                    'dropout_rate': dropout_rate,
                    'epochs_trained': epochs_trained,
                    'history': history
                }, model_path)
                
                plot_filename = f'training_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.png'
                plot_path = os.path.join(plots_dir, plot_filename)
                plot_training_curve(history, plot_path)
                print(f"   Training plot saved")
                
                print(f"\n Performing MC Dropout sampling ({mc_samples} passes)...")
                
                
                mc_predictions_scaled = mc_dropout_predict(model, test_loader, n_samples=mc_samples, verbose=True)
                
                
                y_pred_p10_scaled = np.percentile(mc_predictions_scaled, 10, axis=0)
                y_pred_p50_scaled = np.percentile(mc_predictions_scaled, 50, axis=0)
                y_pred_p90_scaled = np.percentile(mc_predictions_scaled, 90, axis=0)
                
                
                pred_p10 = invTransform(scaler, y_pred_p10_scaled, 'GHI', colNames)
                pred_p50 = invTransform(scaler, y_pred_p50_scaled, 'GHI', colNames)
                pred_p90 = invTransform(scaler, y_pred_p90_scaled, 'GHI', colNames)
                test = invTransform(scaler, y_test, 'GHI', colNames)
                
               
                mc_samples_inv = np.zeros_like(mc_predictions_scaled)
                for i in range(mc_samples):
                    mc_samples_inv[i] = invTransform(scaler, mc_predictions_scaled[i], 'GHI', colNames).reshape(-1, 1)
                
                print(f"MC sampling complete")
                
         
                MAPE_P50 = calculate_mape(test, pred_p50)
                RMSE_P50 = np.sqrt(mean_squared_error(test, pred_p50))
                nRMSE_P50 = RMSE_P50 / (np.mean(test) + 1e-8)
                MAE_P50 = mean_absolute_error(test, pred_p50)
                nMAE_P50 = MAE_P50 / (np.mean(test) + 1e-8)
                R2_P50 = r2_score(test, pred_p50)
                MBE_P50 = MBE(test, pred_p50)
                
                quantile_metrics = calculate_quantile_metrics(test, pred_p10, pred_p50, pred_p90)

                

                plot_filename = f'predictions_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.png'
                plot_path = os.path.join(plots_dir, plot_filename)
                plot_mc_predictions(test, pred_p10, pred_p50, pred_p90, mc_samples_inv, plot_path)
                print(f" Prediction plot saved")
  
                plot_filename = f'coverage_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.png'
                plot_path = os.path.join(plots_dir, plot_filename)
                plot_coverage_analysis(test, pred_p10, pred_p50, pred_p90, plot_path)
                print(f"  Coverage plot saved")
                
     
                quantile_predictions_df = pd.DataFrame({
                    'y_true': test,
                    'y_pred_p10': pred_p10,
                    'y_pred_p50': pred_p50,
                    'y_pred_p90': pred_p90,
                    'interval_width': pred_p90 - pred_p10,
                    'in_interval': (test >= pred_p10) & (test <= pred_p90)
                })
                
                quantile_pred_filename = f'quantile_predictions_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.csv'
                quantile_pred_path = os.path.join(report_dir, quantile_pred_filename)
                quantile_predictions_df.to_csv(quantile_pred_path, index=False)
        
                summary_results.append({
                    'samplingFrequency': samplingFrequency,
                    'polynomialAugmentation': polynomialAug,
                    'features': featureColumns_eng_list[k],
                    'history': nPrevSteps,
                    'epochs_trained': epochs_trained,
                    'dropout_rate': dropout_rate,
                    'mc_samples': mc_samples,
                    'MAPE_P50': MAPE_P50,
                    'RMSE_P50': RMSE_P50,
                    'nRMSE_P50': nRMSE_P50,
                    'MAE_P50': MAE_P50,
                    'nMAE_P50': nMAE_P50,
                    'R2_P50': R2_P50,
                    'MBE_P50': MBE_P50,
                    'PICP_80': quantile_metrics['PICP_80'],
                    'PINAW': quantile_metrics['PINAW'],
                    'QuantileLoss_P10': quantile_metrics['QuantileLoss_P10'],
                    'QuantileLoss_P50': quantile_metrics['QuantileLoss_P50'],
                    'QuantileLoss_P90': quantile_metrics['QuantileLoss_P90'],
                    'AvgQuantileScore': quantile_metrics['AvgQuantileScore'],
                    'IntervalWidth': quantile_metrics['IntervalWidth']
                })
                

                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
    

    if len(summary_results) > 0:
        summary_df = pd.DataFrame(summary_results)
        summary_filename = f'summary_mcdropout_{samplingFrequency}.csv'
        summary_path = os.path.join(report_dir, summary_filename)
        summary_df.to_csv(summary_path, index=False)
        print(f"\nSummary saved: {summary_filename}")

        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle(f'MC Dropout Performance Overview: {samplingFrequency}', 
                    fontsize=16, fontweight='bold')
        
        picp_values = summary_df['PICP_80'].values
        mape_values = summary_df['MAPE_P50'].values
        interval_values = summary_df['IntervalWidth'].values
        
        # Graphs
        ax = axes[0, 0]
        ax.hist(picp_values, bins=20, alpha=0.7, edgecolor='black', color='steelblue')
        ax.axvline(x=80, color='red', linestyle='--', linewidth=2, label='Target (80%)')
        ax.axvline(x=picp_values.mean(), color='green', linestyle='--', linewidth=2, 
                  label=f'Mean ({picp_values.mean():.1f}%)')
        ax.axvspan(75, 85, alpha=0.2, color='green', label='Good Range')
        ax.set_xlabel('PICP (%)')
        ax.set_ylabel('Frequency')
        ax.set_title('Coverage Distribution')
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')

        ax = axes[0, 1]
        scatter = ax.scatter(mape_values, picp_values, alpha=0.6, s=100, 
                           c=interval_values, cmap='viridis')
        ax.axhline(y=80, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
        ax.axhspan(75, 85, alpha=0.1, color='green')
        ax.set_xlabel('MAPE (%)')
        ax.set_ylabel('PICP (%)')
        ax.set_title('Accuracy vs Coverage Trade-off')
        ax.grid(True, alpha=0.3)
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label('Interval Width (W/m²)', rotation=270, labelpad=20)
        

        ax = axes[1, 0]
        ax.hist(interval_values, bins=20, alpha=0.7, edgecolor='black', color='coral')
        ax.axvline(x=interval_values.mean(), color='red', linestyle='--', 
                  linewidth=2, label=f'Mean: {interval_values.mean():.1f} W/m²')
        ax.set_xlabel('Average Interval Width (W/m²)')
        ax.set_ylabel('Frequency')
        ax.set_title('Uncertainty Distribution')
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')

        ax = axes[1, 1]
        ax.axis('off')

        summary_df['coverage_error'] = np.abs(summary_df['PICP_80'] - 80)
        summary_df_sorted = summary_df.sort_values('coverage_error')
        
        best_text = "Top 5 configs\n"
        best_text += "="*55 + "\n\n"
        
        for idx, (_, row) in enumerate(summary_df_sorted.head(5).iterrows(), 1):
            best_text += f"{idx}. {row['features']}, "
            best_text += f"Poly={int(row['polynomialAugmentation'])}, "
            best_text += f"Hist={int(row['history'])}\n"
            best_text += f"   PICP: {row['PICP_80']:.1f}%  |  "
            best_text += f"MAPE: {row['MAPE_P50']:.2f}%  |  "
            best_text += f"Width: {row['IntervalWidth']:.1f}\n\n"
        
        best_text += "\n" + "="*55 + "\n"
        best_text += f"Overall Statistics:\n"
        best_text += f"  Avg Coverage: {picp_values.mean():.1f}% ± {picp_values.std():.1f}%\n"
        best_text += f"  Avg MAPE:     {mape_values.mean():.2f}% ± {mape_values.std():.2f}%\n"
        best_text += f"  Configs in 75-85%: {((picp_values >= 75) & (picp_values <= 85)).sum()}/{len(picp_values)}\n"
        
        ax.text(0.05, 0.5, best_text, fontsize=9, family='monospace',
               verticalalignment='center', transform=ax.transAxes,
               bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))
        
        plt.tight_layout()
        overview_path = os.path.join(report_dir, f'overview_{samplingFrequency}.png')
        plt.savefig(overview_path, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"Overview plot saved: overview_{samplingFrequency}.png")


total_time = time.time() - start_time
print("\n" + "="*80)
print("TRAINING COMPLETE")
print("="*80)
print(f"Total time: {total_time/60:.1f} minutes ({total_time/3600:.2f} hours)")
print(f" Results: {reportLSTM_dir}")

Total configurations: 48

Frequency: 15_minutes (1/3)
Loaded: 85620 samples

Features: GHI (1/4)
  [1/48] Poly=1, Hist=10 | ETA: 0.2min

 Training MC Dropout model...


Training:  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍   | 49/50 [05:14<00:06,  6.41s/it, train=0.004495, val=0.004149, best_val=0.004088, patience=9/10]



 Early stopping at epoch 50
 Model trained (50 epochs)
   Training plot saved

 Performing MC Dropout sampling (100 passes)...


MC sampling complete
 Prediction plot saved
  Coverage plot saved
  [2/48] Poly=1, Hist=20 | ETA: 222.3min

 Training MC Dropout model...


Training:  96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊       | 48/50 [05:44<00:14,  7.17s/it, train=0.004750, val=0.004387, best_val=0.004252, patience=9/10]


...

 Model trained (50 epochs)
   Training plot saved

 Performing MC Dropout sampling (100 passes)...


MC sampling complete
 Prediction plot saved
  Coverage plot saved
  [48/48] Poly=2, Hist=20 | ETA: 0.0min

 Training MC Dropout model...


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:59<00:00,  5.98s/it, train=0.010775, val=0.006297, best_val=0.005834, patience=5/10]


 Model trained (50 epochs)
   Training plot saved

 Performing MC Dropout sampling (100 passes)...


MC sampling complete
 Prediction plot saved
  Coverage plot saved

Summary saved: summary_mcdropout_1_hour.csv
Overview plot saved: overview_1_hour.png

TRAINING COMPLETE
Total time: 394.4 minutes (6.57 hours)
 Results: ./reports/LSTM_mcdropout/


# Results are okayish but the model is overfitting so we are trying with some other dropout   

## Target: Get PICP to 70-85% range
-> Validate Calibration:
Once we have ~75-85% coverage, we will verify if it's consistent:

Check coverage across different GHI ranges (low/medium/high irradiance)
Check coverage across seasons (winter vs summer)
Ensure intervals aren't just uniformly wider everywhere

# CONFIGURATION - CHANGES HERE


In [71]:
nbEpoch = 50
batch_size = 512  
learning_rate = 0.001  
patience = 10
mc_samples = 100
dropout_rate = 0.35  
num_workers = 4  

nPrevSteps_list = [10, 20]  
polynomialAugm_list = [1, 2] 
samplingFrequencies_eng = ["15_minutes"]  


featureColumns_list = [
    basicFeatures,                           
    basicFeatures + d1Features,              
    basicFeatures + seasonalFeatures + d1Features  
]
featureColumns_eng_list = ['GHI', 'GHI_d1', 'GHI_season_d1']  


reportLSTM_dir = './reports2/LSTM_mcdropout_calibrated/' 

In [72]:
def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs=50, patience=10, verbose=True):
    """early stopping and progress tracking"""
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    history = {'train_loss': [], 'val_loss': [], 'epoch': []}
    
    use_amp = torch.cuda.is_available()
    if use_amp:
        scaler = torch.cuda.amp.GradScaler()
    
    epoch_pbar = tqdm(range(num_epochs), desc="Training", disable=not verbose)
    
    for epoch in epoch_pbar:
        # Training phase
        model.train()
        train_loss = 0.0
        
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device, non_blocking=True), batch_y.to(device, non_blocking=True)
            
            optimizer.zero_grad(set_to_none=True)
            
            if use_amp:
                with torch.cuda.amp.autocast():
                    outputs = model(batch_x, use_dropout=True)
                    loss = criterion(outputs, batch_y)
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(batch_x, use_dropout=True)
                loss = criterion(outputs, batch_y)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x, use_dropout=False)  
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        history['epoch'].append(epoch + 1)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        
   
        epoch_pbar.set_postfix({
            'train': f'{train_loss:.6f}',
            'val': f'{val_loss:.6f}',
            'best_val': f'{best_val_loss:.6f}',
            'patience': f'{patience_counter}/{patience}'
        })

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\n Early stopping at epoch {epoch+1}")
                break
    
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, len(history['epoch']), history

In [73]:
def mc_dropout_predict(model, data_loader, n_samples=100, verbose=True):
    model.train() 
    first_batch = next(iter(data_loader))
    n_test = sum(len(batch_y) for _, batch_y in data_loader)
    
    all_predictions = torch.zeros(n_samples, n_test, 1, device=device)
    
    sample_pbar = tqdm(range(n_samples), desc="MC Sampling", disable=not verbose, leave=False)
    
    for sample_idx in sample_pbar:
        batch_start = 0
        
        with torch.no_grad():
            for batch_x, _ in data_loader:
                batch_x = batch_x.to(device, non_blocking=True)
                batch_size = batch_x.size(0)
                
                outputs = model(batch_x, use_dropout=True)
                all_predictions[sample_idx, batch_start:batch_start+batch_size] = outputs
                
                batch_start += batch_size
    
    return all_predictions.cpu().numpy()

In [74]:
class MCDropoutLSTM(nn.Module):
    """aggressive dropout for better uncertainty calibration"""
    def __init__(self, input_size, hidden_size=300, num_layers=3, dropout_rate=0.35):  
        super(MCDropoutLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.dropout3 = nn.Dropout(dropout_rate)
        
        self.dropout_fc = nn.Dropout(dropout_rate * 0.8) 
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x, use_dropout=True):
        out, _ = self.lstm1(x)
        if use_dropout:
            out = self.dropout1(out)
        
        out, _ = self.lstm2(out)
        if use_dropout:
            out = self.dropout2(out)
        
        out, _ = self.lstm3(out)
        if use_dropout:
            out = self.dropout3(out)
        
        out = out[:, -1, :]
        
        if use_dropout:
            out = self.dropout_fc(out)
            
        out = self.fc(out)
        return out

In [75]:
total_configs = len(samplingFrequencies_eng) * len(featureColumns_list) * len(polynomialAugm_list) * len(nPrevSteps_list)
current_config = 0
start_time = time.time()

print(f"Total configurations: {total_configs}")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Training 1 model per config (vs 10 for ensemble) → 10x faster!\n")

for freq_idx, samplingFrequency in enumerate(samplingFrequencies_eng):
    print("\n" + "="*80)
    print(f"Frequency: {samplingFrequency} ({freq_idx+1}/{len(samplingFrequencies_eng)})")
    print("="*80)
    
    report_dir = os.path.join(reportLSTM_dir, samplingFrequency)
    models_dir = os.path.join(report_dir, 'models')
    plots_dir = os.path.join(report_dir, 'plots')
    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)
    
    file_path = os.path.join(data_dir, f'GHI_sampled_{samplingFrequency}.csv')
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    
    data_df = pd.read_csv(file_path, index_col=0)
    print(f" Loaded: {data_df.shape[0]} samples")
    
    summary_results = []
    
    for k, featureColumns in enumerate(featureColumns_list):
        print(f"\nFeatures: {featureColumns_eng_list[k]} ({k+1}/{len(featureColumns_list)})")
        
        for polynomialAug in polynomialAugm_list:
            for nPrevSteps in nPrevSteps_list:
                current_config += 1
                elapsed = time.time() - start_time
                eta = (elapsed / current_config) * (total_configs - current_config)
                
                print(f"\n  {'─'*76}")
                print(f"[{current_config}/{total_configs}] Poly={polynomialAug}, Hist={nPrevSteps} | ETA: {eta/60:.1f}min")
                print(f"  {'─'*76}")
                
                set_seed(42)
                X = data_df.loc[:, featureColumns]
                
                poly = PolynomialFeatures(polynomialAug)
                X_poly = poly.fit_transform(X)
                colNames = poly.get_feature_names_out(X.columns)
                
                scaler = MinMaxScaler(feature_range=(0, 1))
                scaled = scaler.fit_transform(X_poly)
                scaled = pd.DataFrame(scaled, columns=colNames, index=data_df.index)
                scaled = scaled.drop("1", axis=1)
                nFeatureColumns = scaled.shape[1]
                
                train_test_df = pd.DataFrame()
                for i in range(nPrevSteps + 1):
                    title = scaled.columns + 't(-' + str(i) + ')'
                    temp = scaled.shift(periods=i)
                    temp.columns = title
                    train_test_df = pd.concat([train_test_df, temp], axis=1)
                
                train_test_df = train_test_df.dropna()
                
                title_0 = scaled.columns + 't(-' + str(0) + ')'
                X = train_test_df.drop(title_0, axis=1).values
                y = train_test_df['GHIt(-0)'].values.reshape(-1, 1)
                
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
                X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=False)
                
                X_train_r = X_train.reshape((X_train.shape[0], nPrevSteps, nFeatureColumns))
                X_val_r = X_val.reshape((X_val.shape[0], nPrevSteps, nFeatureColumns))
                X_test_r = X_test.reshape((X_test.shape[0], nPrevSteps, nFeatureColumns))
                
                train_dataset = TensorDataset(torch.FloatTensor(X_train_r), torch.FloatTensor(y_train))
                val_dataset = TensorDataset(torch.FloatTensor(X_val_r), torch.FloatTensor(y_val))
                test_dataset = TensorDataset(torch.FloatTensor(X_test_r), torch.FloatTensor(y_test))
                
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False,
                                         num_workers=num_workers, pin_memory=True)
                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                                       num_workers=num_workers, pin_memory=True)
                test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                                        num_workers=num_workers, pin_memory=True)

                # Train
                print(f"\nTraining MC Dropout model...")
                
                model = MCDropoutLSTM(
                    input_size=nFeatureColumns,
                    hidden_size=300,
                    num_layers=3,
                    dropout_rate=dropout_rate
                ).to(device)
                
                criterion = nn.MSELoss()
                optimizer = optim.Adam(model.parameters(), lr=learning_rate)
                
                model, epochs_trained, history = train_model(
                    model, train_loader, val_loader,
                    criterion, optimizer, nbEpoch, patience, verbose=True
                )
                
                print(f"Model trained ({epochs_trained} epochs)")
                
                # Save model
                model_filename = f'model_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.pt'
                model_path = os.path.join(models_dir, model_filename)
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'input_size': nFeatureColumns,
                    'hidden_size': 300,
                    'dropout_rate': dropout_rate,
                    'epochs_trained': epochs_trained,
                    'history': history
                }, model_path)
                
                plot_filename = f'training_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.png'
                plot_path = os.path.join(plots_dir, plot_filename)
                plot_training_curve(history, plot_path)
                print(f"Training plot saved")
                

                print(f"\nPerforming MC Dropout sampling ({mc_samples} passes)...")
                
            
                mc_predictions_scaled = mc_dropout_predict(model, test_loader, n_samples=mc_samples, verbose=True)
                
         
                y_pred_p10_scaled = np.percentile(mc_predictions_scaled, 10, axis=0)
                y_pred_p50_scaled = np.percentile(mc_predictions_scaled, 50, axis=0)
                y_pred_p90_scaled = np.percentile(mc_predictions_scaled, 90, axis=0)
   
                pred_p10 = invTransform(scaler, y_pred_p10_scaled, 'GHI', colNames)
                pred_p50 = invTransform(scaler, y_pred_p50_scaled, 'GHI', colNames)
                pred_p90 = invTransform(scaler, y_pred_p90_scaled, 'GHI', colNames)
                test = invTransform(scaler, y_test, 'GHI', colNames)

                mc_samples_inv = np.zeros_like(mc_predictions_scaled)
                for i in range(mc_samples):
                    mc_samples_inv[i] = invTransform(scaler, mc_predictions_scaled[i], 'GHI', colNames).reshape(-1, 1)
                
                print(f" MC sampling complete")
                
                MAPE_P50 = calculate_mape(test, pred_p50)
                RMSE_P50 = np.sqrt(mean_squared_error(test, pred_p50))
                nRMSE_P50 = RMSE_P50 / (np.mean(test) + 1e-8)
                MAE_P50 = mean_absolute_error(test, pred_p50)
                nMAE_P50 = MAE_P50 / (np.mean(test) + 1e-8)
                R2_P50 = r2_score(test, pred_p50)
                MBE_P50 = MBE(test, pred_p50)
                
                quantile_metrics = calculate_quantile_metrics(test, pred_p10, pred_p50, pred_p90)
                
      
                
                plot_filename = f'predictions_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.png'
                plot_path = os.path.join(plots_dir, plot_filename)
                plot_mc_predictions(test, pred_p10, pred_p50, pred_p90, mc_samples_inv, plot_path)
                print(f" Prediction plot saved")

                plot_filename = f'coverage_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.png'
                plot_path = os.path.join(plots_dir, plot_filename)
                plot_coverage_analysis(test, pred_p10, pred_p50, pred_p90, plot_path)
                print(f" Coverage plot saved")
                
                quantile_predictions_df = pd.DataFrame({
                    'y_true': test,
                    'y_pred_p10': pred_p10,
                    'y_pred_p50': pred_p50,
                    'y_pred_p90': pred_p90,
                    'interval_width': pred_p90 - pred_p10,
                    'in_interval': (test >= pred_p10) & (test <= pred_p90)
                })
                
                quantile_pred_filename = f'quantile_predictions_{samplingFrequency}_{featureColumns_eng_list[k]}_P{polynomialAug}_H{nPrevSteps}.csv'
                quantile_pred_path = os.path.join(report_dir, quantile_pred_filename)
                quantile_predictions_df.to_csv(quantile_pred_path, index=False)
                
                summary_results.append({
                    'samplingFrequency': samplingFrequency,
                    'polynomialAugmentation': polynomialAug,
                    'features': featureColumns_eng_list[k],
                    'history': nPrevSteps,
                    'epochs_trained': epochs_trained,
                    'dropout_rate': dropout_rate,
                    'mc_samples': mc_samples,
                    'MAPE_P50': MAPE_P50,
                    'RMSE_P50': RMSE_P50,
                    'nRMSE_P50': nRMSE_P50,
                    'MAE_P50': MAE_P50,
                    'nMAE_P50': nMAE_P50,
                    'R2_P50': R2_P50,
                    'MBE_P50': MBE_P50,
                    'PICP_80': quantile_metrics['PICP_80'],
                    'PINAW': quantile_metrics['PINAW'],
                    'QuantileLoss_P10': quantile_metrics['QuantileLoss_P10'],
                    'QuantileLoss_P50': quantile_metrics['QuantileLoss_P50'],
                    'QuantileLoss_P90': quantile_metrics['QuantileLoss_P90'],
                    'AvgQuantileScore': quantile_metrics['AvgQuantileScore'],
                    'IntervalWidth': quantile_metrics['IntervalWidth']
                })
                
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
    

    if len(summary_results) > 0:
        summary_df = pd.DataFrame(summary_results)
        summary_filename = f'summary_mcdropout_{samplingFrequency}.csv'
        summary_path = os.path.join(report_dir, summary_filename)
        summary_df.to_csv(summary_path, index=False)
        print(f"\nSummary saved: {summary_filename}")
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle(f'MC Dropout Performance Overview: {samplingFrequency}', 
                    fontsize=16, fontweight='bold')
        
        picp_values = summary_df['PICP_80'].values
        mape_values = summary_df['MAPE_P50'].values
        interval_values = summary_df['IntervalWidth'].values

        ax = axes[0, 0]
        ax.hist(picp_values, bins=20, alpha=0.7, edgecolor='black', color='steelblue')
        ax.axvline(x=80, color='red', linestyle='--', linewidth=2, label='Target (80%)')
        ax.axvline(x=picp_values.mean(), color='green', linestyle='--', linewidth=2, 
                  label=f'Mean ({picp_values.mean():.1f}%)')
        ax.axvspan(75, 85, alpha=0.2, color='green', label='Good Range')
        ax.set_xlabel('PICP (%)')
        ax.set_ylabel('Frequency')
        ax.set_title('Coverage Distribution')
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
        

        ax = axes[0, 1]
        scatter = ax.scatter(mape_values, picp_values, alpha=0.6, s=100, 
                           c=interval_values, cmap='viridis')
        ax.axhline(y=80, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
        ax.axhspan(75, 85, alpha=0.1, color='green')
        ax.set_xlabel('MAPE (%)')
        ax.set_ylabel('PICP (%)')
        ax.set_title('Accuracy vs Coverage Trade-off')
        ax.grid(True, alpha=0.3)
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label('Interval Width (W/m²)', rotation=270, labelpad=20)
        
        ax = axes[1, 0]
        ax.hist(interval_values, bins=20, alpha=0.7, edgecolor='black', color='coral')
        ax.axvline(x=interval_values.mean(), color='red', linestyle='--', 
                  linewidth=2, label=f'Mean: {interval_values.mean():.1f} W/m²')
        ax.set_xlabel('Average Interval Width (W/m²)')
        ax.set_ylabel('Frequency')
        ax.set_title('Uncertainty Distribution')
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
        
        ax = axes[1, 1]
        ax.axis('off')
        
        summary_df['coverage_error'] = np.abs(summary_df['PICP_80'] - 80)
        summary_df_sorted = summary_df.sort_values('coverage_error')
        
        best_text = "TOP 5 CONFIGURATIONS\n"
        best_text += "="*55 + "\n\n"
        
        for idx, (_, row) in enumerate(summary_df_sorted.head(5).iterrows(), 1):
            best_text += f"{idx}. {row['features']}, "
            best_text += f"Poly={int(row['polynomialAugmentation'])}, "
            best_text += f"Hist={int(row['history'])}\n"
            best_text += f"   PICP: {row['PICP_80']:.1f}%  |  "
            best_text += f"MAPE: {row['MAPE_P50']:.2f}%  |  "
            best_text += f"Width: {row['IntervalWidth']:.1f}\n\n"
        
        best_text += "\n" + "="*55 + "\n"
        best_text += f"Overall Statistics:\n"
        best_text += f"  Avg Coverage: {picp_values.mean():.1f}% ± {picp_values.std():.1f}%\n"
        best_text += f"  Avg MAPE:     {mape_values.mean():.2f}% ± {mape_values.std():.2f}%\n"
        best_text += f"  Configs in 75-85%: {((picp_values >= 75) & (picp_values <= 85)).sum()}/{len(picp_values)}\n"
        
        ax.text(0.05, 0.5, best_text, fontsize=9, family='monospace',
               verticalalignment='center', transform=ax.transAxes,
               bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))
        
        plt.tight_layout()
        overview_path = os.path.join(report_dir, f'overview_{samplingFrequency}.png')
        plt.savefig(overview_path, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"Overview plot saved: overview_{samplingFrequency}.png")

total_time = time.time() - start_time
print("\n" + "="*80)
print("MC DROPOUT changed training complete")
print("="*80)
print(f"Total time: {total_time/60:.1f} minutes ({total_time/3600:.2f} hours)")
print(f" Results: {reportLSTM_dir}")

Total configurations: 12
Started: 2025-11-29 09:34:41
Training 1 model per config (vs 10 for ensemble) → 10x faster!


Frequency: 15_minutes (1/1)
 Loaded: 85620 samples

Features: GHI (1/3)

  ────────────────────────────────────────────────────────────────────────────
[1/12] Poly=1, Hist=10 | ETA: 0.1min
  ────────────────────────────────────────────────────────────────────────────


C:\Users\atafr\AppData\Local\Temp\ipykernel_40308\1284388874.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



Training MC Dropout model...


Training:   0%|                                                                                                                                                                                                                                                            | 0/50 [00:00<?, ?it/s]C:\Users\atafr\AppData\Local\Temp\ipykernel_40308\1284388874.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [04:23<00:00,  5.26s/it, train=0.004739, val=0.004131, best_val=0.004165, patience=2/10]


Model trained (50 epochs)
Training plot saved

Performing MC Dropout sampling (100 passes)...


...

C:\Users\atafr\AppData\Local\Temp\ipykernel_40308\1284388874.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



Training MC Dropout model...


Training:   0%|                                                                                                                                                                                                                                                            | 0/50 [00:00<?, ?it/s]C:\Users\atafr\AppData\Local\Temp\ipykernel_40308\1284388874.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training:  62%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                    | 31/50 [03:00<01:50,  5.84s/it, train=0.006592, val=0.006118, best_val=0.004465, patience=9/10]



 Early stopping at epoch 32
Model trained (32 epochs)
Training plot saved

Performing MC Dropout sampling (100 passes)...


 MC sampling complete
 Prediction plot saved
 Coverage plot saved

Summary saved: summary_mcdropout_15_minutes.csv
Overview plot saved: overview_15_minutes.png

MC DROPOUT changed training complete
Total time: 90.3 minutes (1.51 hours)
 Results: ./reports2/LSTM_mcdropout_calibrated/
